# 1.beautifulsoup의 parsing하는 여러 가지 함수 사용

## 1.1 select 사용하기

In [79]:
from bs4 import BeautifulSoup 

html = '''
<ol>
    <li>NEVER - 국민의 아들</li> 
    <li>SIGNAL - TWICE</li> 
    <li>LONELY - 씨스타</li> 
    <li>I LUV IT - PSY</li> 
    <li>New Face - PSY</li>
</ol>
'''

html

'\n<ol>\n    <li>NEVER - 국민의 아들</li> \n    <li>SIGNAL - TWICE</li> \n    <li>LONELY - 씨스타</li> \n    <li>I LUV IT - PSY</li> \n    <li>New Face - PSY</li>\n</ol>\n'

In [80]:
html = '''
<ol>
    <li id='na' class='first'>NEVER - 국민의 아들</li> 
    <li id='tw' class='first'>SIGNAL - TWICE</li> 
    <li id='lo' class='second'>LONELY - 씨스타</li> 
    <li id='lu' class='second'>I LUV IT - PSY</li> 
    <li>New Face - PSY</li>
</ol>
'''

soup = BeautifulSoup(html, 'html.parser')
print(soup.select('li'))
print(type(soup.select('li')))
print(soup.select('li')[1])
print(soup.select_one('li'))
print(type(soup.select_one('li')))

[<li class="first" id="na">NEVER - 국민의 아들</li>, <li class="first" id="tw">SIGNAL - TWICE</li>, <li class="second" id="lo">LONELY - 씨스타</li>, <li class="second" id="lu">I LUV IT - PSY</li>, <li>New Face - PSY</li>]
<class 'list'>
<li class="first" id="tw">SIGNAL - TWICE</li>
<li class="first" id="na">NEVER - 국민의 아들</li>
<class 'bs4.element.Tag'>


In [81]:
# tag명이 li 이고, class 이름이  'first'인 tag를 가져오기
for tag in soup.select('li.first'):
    print(tag.string)

NEVER - 국민의 아들
SIGNAL - TWICE


In [82]:
# tag의 id값이 'tw'인 tag 가져오기
print(soup.select_one('#tw').text)
print(soup.select('#tw')[0].string)

SIGNAL - TWICE
SIGNAL - TWICE


In [83]:
# li tag 중에서 5번째 li tag 가져오기
soup.select_one('li:nth-of-type(5)').text

'New Face - PSY'

## 1.2 fine(), find_all() Method 사용하기

In [84]:
print(soup.find_all('li'))
print(type(soup.find_all('li')))

# tag명이 li이고, class 이름이 'second'인 tag를 가져오기
for tag in soup.find_all('li', attrs={'class':'second'}):
    print(tag.string)

[<li class="first" id="na">NEVER - 국민의 아들</li>, <li class="first" id="tw">SIGNAL - TWICE</li>, <li class="second" id="lo">LONELY - 씨스타</li>, <li class="second" id="lu">I LUV IT - PSY</li>, <li>New Face - PSY</li>]
<class 'bs4.element.ResultSet'>
LONELY - 씨스타
I LUV IT - PSY


In [85]:
print(soup.find('li', attrs={'class':'second', 'id':'lo'}).string)

LONELY - 씨스타


# 2. 기상청 날씨 데이터 파싱하기 (find, find_all 사용)

In [86]:
import requests
from bs4 import BeautifulSoup
url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"

![기상청 데이터](./images/17.png)

In [87]:
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
# soup

![네이버 웹툰](./images/18.png)

In [88]:
title = soup.find('header').find('title').text # 부분적인 값 가져 오기 
datas = soup.find('location', attrs={'wl_ver':'3'}).find_all('data')
for data in datas:
    print(data.find('mode').string + ' ' + data.find('tmef').string + ' '+ data.find('reliability').text)

A02 2018-06-17 00:00 보통
A02 2018-06-17 12:00 보통
A02 2018-06-18 00:00 보통
A02 2018-06-18 12:00 보통
A02 2018-06-19 00:00 보통
A02 2018-06-19 12:00 보통
A02 2018-06-20 00:00 보통
A02 2018-06-20 12:00 보통
A02 2018-06-21 00:00 보통
A02 2018-06-21 12:00 보통
A01 2018-06-22 00:00 보통
A01 2018-06-23 00:00 보통
A01 2018-06-24 00:00 보통


# 3.Melon 100곡 데이터 파싱하기 (select, select_on 사용)

### 아래 3가지 정보를 알고 시작하자

![멜론 사이트](./images/19.png)

![앨범 정보](./images/20.png)

### User-Agent (안 넣어 주면 값을 가져 오지 못한다.)

![user-id 값](./images/21.png)

In [89]:
import requests
from bs4 import BeautifulSoup
import re

In [90]:
headers = { # 메론 사이트에 꼭들어 가야 하는 내용
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
}

In [91]:
url = 'https://www.melon.com/chart/day/index.htm'

In [92]:
html = requests.get(url, headers=headers).text
soup = BeautifulSoup(html, 'html.parser')

![긁어온 내용 처리 하기](./images/23.png)

In [93]:
# song list 가져오기
song_list = soup.select('#tb_list tr .wrap_song_info a[href*=playSong]')

### song list 만 가져 와서 정리 한 내용

![정리한 내용 보여 주기](./images/24.png)

### 정규 표현식 확인 가능한 Site : https://regexr.com/  뒤쪽 Song Id를 가져 오기

![정규표현식으로 처리 하기](./images/22.png)

In [68]:
for idx, song in enumerate(song_list, 1):
    title = song.text
    link = song['href']
    matched = re.search(r"(\d+)\);", link) # 정규 표현식 처리 
    
    if matched:
        songId = matched.group(1)
        songUrl = 'https://www.melon.com/song/detail.htm?songId='+ songId # song 정보 가져 오기
        print(idx, title, songUrl)

1 여행 https://www.melon.com/song/detail.htm?songId=31093710
2 FAKE LOVE https://www.melon.com/song/detail.htm?songId=31085237
3 지나오다 https://www.melon.com/song/detail.htm?songId=30699142
4 밤 (Time for the moon night) https://www.melon.com/song/detail.htm?songId=31052421
5 주지마 https://www.melon.com/song/detail.htm?songId=31035061
6 빙글뱅글 (Bingle Bangle) https://www.melon.com/song/detail.htm?songId=31098083
7 You https://www.melon.com/song/detail.htm?songId=30849733
8 켜줘 (Light) https://www.melon.com/song/detail.htm?songId=31113240
9 모든 날, 모든 순간 (Every day, Every Moment) https://www.melon.com/song/detail.htm?songId=30962526
10 사랑을 했다 (LOVE SCENARIO) https://www.melon.com/song/detail.htm?songId=30859584
11 What is Love? https://www.melon.com/song/detail.htm?songId=31006730
12 별이 빛나는 밤 https://www.melon.com/song/detail.htm?songId=30937275
13 캥거루 (Kangaroo) (Prod. ZICO) https://www.melon.com/song/detail.htm?songId=31113241
14 붕붕 (Feat. 식케이) (Prod. GroovyRoom) https://www.melon.com/song/detail